Github code from authors stored in picrispr-main

TODO:

1. Preprocess data (dir is offtarget_260520_nuc.csv.zip (crisprsql dataset)).
we can use the code from load_data.py and encoding.py.
the author calls the functions from those classes in picrispr.py.

2. Run models (models.py)

3. Test model (test_input.csv)

In [1]:
#Load files
from google.colab import drive
drive.mount('/content/gdrive')
%cd /content/gdrive/.shortcut-targets-by-id/1xWx4iaOa4dZLKNbO8SfeXGyQyzJoKG1L/piCrispr2.0/picrispr-main
!ls

Mounted at /content/gdrive
/content/gdrive/.shortcut-targets-by-id/1xWx4iaOa4dZLKNbO8SfeXGyQyzJoKG1L/piCrispr2.0/picrispr-main
 default_vals   nontransformed.png	      'picrispr data format.gsheet'   test_input.csv
 encoding.py    offtarget_260520_nuc.csv.zip   picrispr.py		      transformedback.png
 load_data.py  'output(1).gsheet'	       __pycache__		     'WillB (1).py'
 models         output.csv		       README.md		      WillB.py
 models.py      output_Option3RNN.csv	       requirements.txt


In [ ]:
#!python WillB.py test_input.csv 3
!python picrispr.py test_input.csv 2 models False False
#picrispr.py 588 throw no such file or directory when running torch models
#Searches for 'models/trainresult_torch_interface_type_s2_class.pickle'
#Directory has 'models/trainresult_tf_interface_type_s2_class.pickle'
#Forcing torch version to look for tf file instead gives error... (screenshot2) and (screenshot4) if mode is changed in load function


In [ ]:
!ls
#load data preproces here

In [ ]:
import pandas as pd
filePath = "offtarget_260520_nuc.csv.zip"
df = pd.read_csv(filePath, low_memory=False)
print(df.columns)

In [ ]:
import sys
from encoding import oneHotSingleNuclTargetMismatchType, oneHotSingleNuclTargetMismatch, oneHotSingleNucl, normaliseCF
from models import mySequential, vecToMatEncoder, vecToMatEncoding
import pickle
from tqdm import tqdm
import xgboost as xgb
import torch
import tensorflow as tf
from scipy.stats import spearmanr
import os
os.environ['TF_FORCE_GPU_ALLOW_GROWTH'] = 'true'
from picrispr import CSVDataset


use_gpu = True

filenames = ["xgboost_interface_type_epi", "torch_eng", "torch_engnuc", "torch_interface_type_nuc", "tf_eng", "tf_engnuc"]
modelnames =["XGB_S3E2", "CNN_S2E0", "CNN_S4E0", "CNN_S5E2", "RNN_S2E3", "RNN_S4E3"]

# define models
seq_features = "target_sequence, grna_target_sequence, "
energy_features = "energy_2+energy_1-(energy_3*energy_4/energy_2), "
epigen_features = "epigen_ctcf, epigen_dnase, epigen_rrbs, epigen_h3k4me3, epigen_drip, "
nuc_features = "NucleotideBDM, NuPoP_Affinity_147_human, GCContent, "
numBpWise = 3

seq_energy_feat         = seq_features + energy_features + "experiment_id"
seq_energy_epi_feat     = seq_features + energy_features + epigen_features + "experiment_id"
seq_energy_nuc_feat     = seq_features + nuc_features + energy_features + "experiment_id"
seq_energy_epi_nuc_feat = seq_features + nuc_features + energy_features + epigen_features + "experiment_id"

# retrieve user input
filePath = "offtarget_260520_nuc.csv.zip"
modelNum   = 3
home       = "models"
regression = False
replace_all_nans = False

args = [("", home, "torch", False, regression, "s4", False), # E3+CRISPRNetStyle, CNN, s4
        ("", home, "torch", False, regression, "s2", False), # E3+CRISPRNetStyle, CNN, s2
        ("", home, "tf",    False, regression, "s4", False), # E3+CRISPRNetStyle, RNN, s4
        ("", home, "tf",    False, regression, "s2", False), # E3+CRISPRNetStyle, RNN, s2
        ("", home, "torch", True,  regression, "s4", True),  # E2,                CNN, s4
        ("", home, "torch", True,  regression, "s2", True),  # E2,                CNN, s2
        ("", home, "tf",    True,  regression, "s4", True),  # E2,                RNN, s4
        ("", home, "tf",    True,  regression, "s2", True),  # E2,                RNN, s2
        ]

kwargs = [{'dbFields': seq_energy_nuc_feat, 'numBpWise': numBpWise, 'CRISPRNetStyle': True},
          {'dbFields': seq_energy_feat,     'numBpWise': 0,         'CRISPRNetStyle': True},
          {'dbFields': seq_energy_nuc_feat, 'numBpWise': numBpWise, 'CRISPRNetStyle': True},
          {'dbFields': seq_energy_feat,     'numBpWise': 0,         'CRISPRNetStyle': True},
          {'dbFields': seq_energy_nuc_feat, 'numBpWise': numBpWise, 'CRISPRNetStyle': False},
          {'dbFields': seq_energy_feat,     'numBpWise': 0,         'CRISPRNetStyle': False},
          {'dbFields': seq_energy_nuc_feat, 'numBpWise': numBpWise, 'CRISPRNetStyle': False},
          {'dbFields': seq_energy_feat,     'numBpWise': 0,         'CRISPRNetStyle': False}
          ]

# load user dataset and chosen model
dataset = CSVDataset(filePath)

config = {"dbFields": kwargs[modelNum]['dbFields'],
          "numBpWise": kwargs[modelNum]['numBpWise'],
          "mode": args[modelNum][2],
          "mismatchType": args[modelNum][3],
          "interfaceMode": args[modelNum][6],
          "chooseSpecies": ["hg19", "hg38"],
          "regression": args[modelNum][4],
          "CRISPRNetStyle": kwargs[modelNum]['CRISPRNetStyle'],
          }

print(config)

filenameAppendix  = "_interface" if config["interfaceMode"] else ""
filenameAppendix += "_type"      if config["mismatchType"] and config["interfaceMode"]  else ""
filenameAppendix += "_"+args[modelNum][5]

print(config["mode"], filenameAppendix, "__________________________________")

if config["mismatchType"]:
    oneHotFct = oneHotSingleNuclTargetMismatchType
    featurenames = ['A_match', 'T_match', 'C_match', 'G_match',
                    'A_mismT', 'T_mismC', 'C_mismG', 'G_mismA',
                    'A_mismC', 'T_mismG', 'C_mismA', 'G_mismT',
                    'A_mismG', 'T_mismA', 'C_mismT', 'G_mismC']
elif config["interfaceMode"]:
    oneHotFct = oneHotSingleNuclTargetMismatch
    featurenames = ['A', 'A_mism', 'T', 'T_mism', 'C', 'C_mism', 'G', 'G_mism']
else:
    oneHotFct = oneHotSingleNucl
    featurenames = ['A',           'T',           'C',           'G']

featurenames.extend(" ".join(config["dbFields"].split()).split(', ')[2:]) # append whatever database fields apart from guide and target sequence are used


# get data matrix from user input
filenameAppendix += "_class" if not config["regression"] else ""

print("encoding dataset...", end = '')
dm = dataset.getDataMatrix(config["dbFields"], oneHotFct, normaliseCF, chooseSpecies=config["chooseSpecies"], filenameAppendix=filenameAppendix,
                            featurenames=featurenames, mode=config["mode"], numBpWise=config["numBpWise"], test_size=0.2, doSplit=True,
                            CRISPRNetStyle=config["CRISPRNetStyle"], replace_all_nans=replace_all_nans)
print("done")

print("config[mode] is ", config["mode"], " so dm.mode was set to be ", dm.mode)
dm.mode = config["mode"]
dm.interfaceMode = config["interfaceMode"]
dm.mismatchType = config["mismatchType"]
dm.regression = config["regression"]
dm.CRISPRNetStyle = config["CRISPRNetStyle"]

# load model
print("loading model...", end = '')
#result = TrainResult.load(home, config["mode"], filenameAppendix, device="gpu:0" if use_gpu else "cpu")
print("done")

# predict on ext set
isHPC = True
bs = int(7e4) if isHPC or dm.mode != "torch" else 35000

# don't use experiment_id column
dm.toDense()
dm = dm.dropColumn(-1)

print("preparing dataset...", end = '')
trainDataset, validDataset, extDataset = dm.prepareDataset(cutoff_class=-4, addGaussian=False)
dm.prepareDataloaders(trainDataset, validDataset, extDataset, bs, balanceClasses=False, ignoreExtSet=True, doSampling=False)
print("done")

TwoLayerNet: 91.56% Accuracy

In [ ]:
from models import TwoLayerNet, ThreeLayerNet, LinearRegressionModel, ConvolutionalNet
import torch.nn as nn
import torch.optim as optim
import numpy as np

#for regression
# lossf = nn.MSELoss()
#for classification
lossf = nn.BCELoss()
model = ConvolutionalNet(regression = False)
# model = ThreeLayerNet(139, 256, 1, regression=True)
# model = TwoLayerNet(139, 256, 1, regression=True)
#model = LinearRegressionModel(139, 256) RuntimeError: shape '[1, 60000]' is invalid for input of size 15360000
optimizer = optim.Adam(model.parameters(), lr=0.001)


num_epochs = 100
for epoch in range(num_epochs):

    model.train()
    for batch in dm.trainLoader:
        inputs, targets, extra = batch
        print(tf.shape(inputs))

        # Forward pass
        outputs = model(inputs)

        # Compute loss
        loss = lossf(outputs, targets)

        # Backward pass and optimization
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

    print(f'Epoch [{epoch+1}/{num_epochs}], Train Loss: {loss.item():.4f}')

    model.eval()  # Set the model to evaluation mode
    total_correct = 0
    total_samples = 0

    with torch.no_grad():  # Disable gradient computation during validation
        for batch in dm.testLoader:
            inputs, targets, extra = batch
            outputs = model(inputs)

            pred = outputs.numpy()
            labels = targets.numpy()


            # Assuming labels are in the range [0, 1] (0 for class 0, 1 for class 1)
            #labels = labels.float().view(-1, 1)  # Ensure labels are of type float and have shape (batch_size, 1)

            # Compute accuracy
            predictions = (pred > 0.5)  # Threshold at 0.5 for binary classification

            correct = np.sum(predictions == labels.astype(int))
            total_correct += correct
            total_samples += len(labels)

    accuracy = total_correct / total_samples
    print(f'Epoch [{epoch+1}/{num_epochs}], Accuracy: {accuracy * 100:.2f}%')



tf.Tensor([60000   139], shape=(2,), dtype=int32)


RuntimeError: ignored